In [12]:
from dotenv import load_dotenv
import os
from mistralai import Mistral
import requests
import numpy as np
import faiss # vector database
load_dotenv()

True

In [5]:
url = "https://www.parliament.go.ke/sites/default/files/2017-05/The_Constitution_of_Kenya_2010.pdf"
response = requests.get(url)
text = response.text
# print(text)

In [7]:
chunk_size = 3000 # split text into chunks
chunks = [text[i:i+chunk_size] for i in range (0, len(text), chunk_size)]
len(chunks)

543

In [13]:
api_key = os.environ["mistral_api_key"]
client = Mistral(api_key = api_key)

In [14]:
def get_single_embedding(text):
    response = client.embeddings.create(
        model="mistral-embed",
        inputs=[text]  # must be a list
    )
    return response.data[0].embedding

In [21]:
import time

def get_embeddings_in_batches(chunks, max_batch_size=50, max_retries=3, sleep_time=2):
    all_embeddings = []

    for i in range(0, len(chunks), max_batch_size):
        batch = chunks[i:i+max_batch_size]

        for attempt in range(max_retries):
            try:
                response = client.embeddings.create(
                    model="mistral-embed",
                    inputs=batch
                )
                batch_embeddings = [item.embedding for item in response.data]
                all_embeddings.extend(batch_embeddings)
                break  # success → break retry loop
            except Exception as e:
                if "rate limit" in str(e).lower():
                    print(f"Rate limit hit. Sleeping {sleep_time}s...")
                    time.sleep(sleep_time)
                    sleep_time *= 2  # exponential backoff
                else:
                    raise

    return all_embeddings

In [23]:
# 1. Create embeddings for your document chunks
text_embeddings = get_embeddings_in_batches(chunks, max_batch_size=10)
text_embeddings = np.array(text_embeddings, dtype="float32")
# 2. Build FAISS index
d = text_embeddings.shape[1]   # embedding dimension
index = faiss.IndexFlatL2(d)   # L2 distance index
index.add(text_embeddings)     # add all chunk embeddings

In [24]:
# Create embedding for the query
question = "What are the Principles of land policy in the constitution of Kenya"
question_embeddings = get_single_embedding(question)
# Convert to numpy and reshape
question_embeddings = np.array(question_embeddings, dtype="float32").reshape(1, -1)
# Search top-k similar chunks

D, I = index.search(question_embeddings, k=2)
# Retrieve the corresponding chunks
retrieved_chunks = [chunks[i] for i in I[0]]

In [25]:
# Prompt Engineering
prompt = f""""
The context information is as below:
{retrieved_chunks}
Given the context information and without prior knowledge answer the query
query: {question}
answer:
"""

In [26]:
def run_mistral(user_message, model = "mistral-large-latest"):
    messages = [
        {
            "role": "user",
            "content": user_message
        }
    ]
    chat_response = client.chat.complete(model=model, messages=messages)

    return chat_response.choices[0].message.content

In [27]:
print(run_mistral(prompt))

The provided context appears to be corrupted or encoded in an unreadable format (likely binary or garbled text), making it impossible to extract meaningful information about the **Principles of Land Policy in the Constitution of Kenya**.

However, based on **general knowledge** (not derived from the given context), here are the key principles of land policy as outlined in **Article 60 of the Constitution of Kenya (2010)**:

### **Principles of Land Policy in Kenya (Article 60)**
1. **Equitable Access** – Land should be held, used, and managed in a way that ensures equitable access for all Kenyans.
2. **Security of Land Rights** – Land rights (including customary, communal, and private ownership) must be protected and secured.
3. **Sustainable and Productive Use** – Land should be used sustainably to support present and future generations.
4. **Transparent and Cost-Effective Administration** – Land administration (allocation, registration, and transfer) must be transparent, efficient, a

In [28]:
# The workflow that makes it easier to ask subsequent questions with just a function
def ask_question(question):
    # embed
    q_emb = get_single_embedding(question)
    q_emb = np.array(q_emb, dtype="float32").reshape(1, -1)
    # search
    D, I = index.search(q_emb, k=2)
    retrieved_chunks = [chunks[i] for i in I[0]]
    # build prompt
    context = "\n".join(retrieved_chunks)
    prompt = f"Answer the question using the context below:\n\nContext:\n{context}\n\nQuestion: {question}\nAnswer:"
    # run model
    return run_mistral(prompt)

In [29]:
print(ask_question("What is the role of the Judiciary in Kenya"))

The provided context appears to be corrupted or unreadable, so I cannot extract meaningful information from it. However, I can answer your question about the **role of the Judiciary in Kenya** based on general knowledge and Kenya's legal framework:

### **Role of the Judiciary in Kenya**
The Judiciary in Kenya is an independent arm of government established under **Article 159 of the Constitution of Kenya (2010)**. Its primary roles include:

1. **Administration of Justice**
   - Interpreting and applying the law fairly and impartially.
   - Resolving disputes between individuals, institutions, and the government.

2. **Upholding the Constitution**
   - Ensuring that all laws, actions, and policies comply with the **Constitution of Kenya (2010)**.
   - Protecting fundamental rights and freedoms (e.g., through **constitutional petitions**).

3. **Judicial Review**
   - Reviewing the constitutionality of laws and executive actions.
   - Declaring laws or government actions **unconstituti

In [30]:
print(ask_question("What does the Road Traffic Act state in the constitution of Kenya?"))

The provided context is a **fragment of a PDF file's internal structure** (showing page objects with metadata like `MediaBox`, `Resources`, and `Group` for transparency). It does **not** contain any textual content from the **Road Traffic Act of Kenya** or the **Constitution of Kenya**.

To answer your question accurately, here’s what the **Road Traffic Act (Cap. 403)** and the **Constitution of Kenya (2010)** state regarding road traffic:

### **1. Road Traffic Act (Cap. 403) – Key Provisions**
The **Road Traffic Act** is the primary legislation governing road use, licensing, and traffic offenses in Kenya. Some key provisions include:
- **Licensing of Drivers & Vehicles**: Requires valid driving licenses and vehicle registration.
- **Traffic Rules**: Covers speed limits, overtaking, pedestrian rights, and road signs.
- **Offenses & Penalties**: Includes penalties for drunk driving, reckless driving, and failure to comply with traffic rules.
- **Insurance Requirements**: Mandates third

In [31]:
print(ask_question("What does the law of Kenya say about the legal drinking age?"))

The provided context appears to be corrupted or unreadable, so I cannot extract any meaningful information about Kenya's laws on the legal drinking age.

However, based on **general knowledge**:
In **Kenya**, the legal drinking age is **18 years old**. This is outlined in the **Alcoholic Drinks Control Act (2010)**, which prohibits the sale or supply of alcohol to anyone under 18. Some counties may have stricter regulations, such as banning alcohol sales near schools or limiting operating hours for bars.

For official legal advice, consult Kenya’s **Alcoholic Drinks Control Act** or local county bylaws.


In [32]:
print(ask_question("What does the law of Kenya say about children's rights"))

The provided context appears to be corrupted or unreadable, so I cannot extract meaningful information about **Kenya's laws on children's rights** from it. However, I can summarize the key legal protections for children in Kenya based on general knowledge:

### **Children’s Rights in Kenya (Key Legal Frameworks)**
Kenya has robust legal protections for children, primarily governed by:

1. **The Constitution of Kenya (2010)**
   - **Article 53** guarantees children’s rights, including:
     - The right to a name and nationality.
     - Free and compulsory basic education.
     - Basic nutrition, shelter, and healthcare.
     - Protection from abuse, neglect, harmful cultural practices, and exploitation.
     - Protection from child labor and early marriage.
     - Access to parental care and protection.

2. **Children’s Act (2022)**
   - Replaced the **Children’s Act (2001)** and aligns with the Constitution and international treaties (e.g., the **UN Convention on the Rights of the Chil

In [33]:
print(ask_question("What does the law of Kenya say sexual offenders Act?"))

The context you provided is a snippet of a **PDF file structure** (likely a table of contents or page objects), not legal text. It does **not** contain any information about Kenya's **Sexual Offences Act**.

For your question about Kenya's **Sexual Offences Act**, here’s a summary of key provisions under the **Sexual Offences Act (No. 3 of 2006)** and related laws:

### **Key Provisions of Kenya’s Sexual Offences Act (2006)**
1. **Definition of Sexual Offences**
   - The Act defines offences such as **rape, defilement, sexual assault, incest, child pornography, sexual harassment, and indecent acts**.
   - **Rape** (Section 3) is defined as intentional, unlawful sexual penetration without consent.
   - **Defilement** (Section 8) involves sexual acts with a minor (under 18), with stricter penalties for younger victims.

2. **Consent**
   - Consent must be **freely given** and cannot be assumed from silence or submission.
   - Consent is **invalid** if obtained through threats, coercion, 